In [7]:
import os
import pandas as pd
import shutil

In [8]:
def read_origin_data():
    dev_df = pd.read_csv('./origin/devset/posts.txt',sep='\t',encoding='utf-8')
    dev_df = dev_df.rename(columns={
        'image_id(s)': 'image_id'
    })
    cols = list(dev_df.columns)
    cols[3], cols[4] = cols[4], cols[3]
    dev_df = dev_df[cols]
    test_df = pd.read_csv('./origin/testset/posts_groundtruth.txt',sep='\t',encoding='utf-8')
    return dev_df,test_df

dev_df,test_df = read_origin_data()

dev_df.head(10)

,post_id,post_text,user_id,username,image_id,timestamp,label
0,324597532548276224,Don't need feds to solve the #bostonbombing wh...,886672620,SantaCruzShred,"boston_fake_03,boston_fake_35",Wed Apr 17 18:57:37 +0000 2013,fake
1,325145334739267584,PIC: Comparison of #Boston suspect Sunil Tripa...,21992286,Oscar_Wang,boston_fake_23,Fri Apr 19 07:14:23 +0000 2013,fake
2,325152091423248385,I'm not completely convinced that it's this Su...,16428755,jamwil,boston_fake_34,Fri Apr 19 07:41:14 +0000 2013,fake
3,324554646976868352,Brutal lo que se puede conseguir en colaboraci...,303138574,rubenson80,"boston_fake_03,boston_fake_35",Wed Apr 17 16:07:12 +0000 2013,fake
4,324315545572896768,4chan and the bombing. just throwing it out th...,180460772,Slimlenny,boston_fake_15,Wed Apr 17 00:17:06 +0000 2013,fake
5,324581777614180352,4chan thinks they found pictures of the bomber...,46224814,iamyadvinder,boston_fake_08,Wed Apr 17 17:55:00 +0000 2013,fake
6,324665423956176896,"Ola ke ase, investigando las bombas de Boston ...",90735851,rcr866,boston_fake_35,Wed Apr 17 23:27:23 +0000 2013,fake
7,325464125868216321,4chan ThinkTank - Imgur http://t.co/hQt2fhxE48,142785938,GlebGgs,"boston_fake_03,boston_fake_35",Sat Apr 20 04:21:09 +0000 2013,fake
8,325099014355820544,@DLoesch have you seen this? Bomber #2 looks ...,21769179,larrygloverlive,boston_fake_13,Fri Apr 19 04:10:19 +0000 2013,fake
9,325705653026975744,da 4chan think tank BOSTON http://t.co/0ZbKMA...,51410043,IsraelTucker,"boston_fake_03,boston_fake_35",Sat Apr 20 20:20:53 +0000 2013,fake


In [9]:

def get_image_set(image_dir):
    image_set = set(filter(lambda x: not x.endswith('.txt'), os.listdir(image_dir)))
    return {file.split('.')[0] for file in image_set}

def counter_no_image(df,image_set):
    counter_no_image = 0
    counter_has_image = 0
    for s in df['image_id']:
        images = set(s.split(','))
        if bool(images & image_set):
            counter_has_image += 1
        else:
            counter_no_image += 1
    return counter_no_image,counter_has_image



            
counter_no_image(dev_df,get_image_set('./origin/devset/images')),counter_no_image(test_df,get_image_set('./origin/testset/images'))    

((2300, 13329), (1079, 1098))

In [10]:


def move_image_dir(image_dir,target_dir):
    os.makedirs(target_dir, exist_ok=True)
    for file in os.listdir(image_dir):
       if not file.endswith('.txt'):
          shutil.copy(image_dir+'/'+file, target_dir+'/'+file) 
          

move_image_dir('./origin/devset/images','./images')    
move_image_dir('./origin/testset/images','./images')    

In [11]:
def has_image(images,image_set):
    return bool(set(images.split(',')) & image_set)
def filter_no_image(df,image_set):
    return df[
        df['image_id'].apply(lambda x: has_image(x,image_set))
    ]

image_set = get_image_set('./images')
legal_data = pd.concat([
    filter_no_image(dev_df,image_set),
    filter_no_image(test_df,image_set),
])
#legal_data = pd.concat([dev_df,test_df]) #使用随机向量代替
legal_data.head(10)

,post_id,post_text,user_id,username,image_id,timestamp,label
0,324597532548276224,Don't need feds to solve the #bostonbombing wh...,886672620.0,SantaCruzShred,"boston_fake_03,boston_fake_35",Wed Apr 17 18:57:37 +0000 2013,fake
1,325145334739267584,PIC: Comparison of #Boston suspect Sunil Tripa...,21992286.0,Oscar_Wang,boston_fake_23,Fri Apr 19 07:14:23 +0000 2013,fake
2,325152091423248385,I'm not completely convinced that it's this Su...,16428755.0,jamwil,boston_fake_34,Fri Apr 19 07:41:14 +0000 2013,fake
3,324554646976868352,Brutal lo que se puede conseguir en colaboraci...,303138574.0,rubenson80,"boston_fake_03,boston_fake_35",Wed Apr 17 16:07:12 +0000 2013,fake
4,324315545572896768,4chan and the bombing. just throwing it out th...,180460772.0,Slimlenny,boston_fake_15,Wed Apr 17 00:17:06 +0000 2013,fake
5,324581777614180352,4chan thinks they found pictures of the bomber...,46224814.0,iamyadvinder,boston_fake_08,Wed Apr 17 17:55:00 +0000 2013,fake
6,324665423956176896,"Ola ke ase, investigando las bombas de Boston ...",90735851.0,rcr866,boston_fake_35,Wed Apr 17 23:27:23 +0000 2013,fake
7,325464125868216321,4chan ThinkTank - Imgur http://t.co/hQt2fhxE48,142785938.0,GlebGgs,"boston_fake_03,boston_fake_35",Sat Apr 20 04:21:09 +0000 2013,fake
8,325099014355820544,@DLoesch have you seen this? Bomber #2 looks ...,21769179.0,larrygloverlive,boston_fake_13,Fri Apr 19 04:10:19 +0000 2013,fake
9,325705653026975744,da 4chan think tank BOSTON http://t.co/0ZbKMA...,51410043.0,IsraelTucker,"boston_fake_03,boston_fake_35",Sat Apr 20 20:20:53 +0000 2013,fake


In [12]:
def split_truth(df):
    return df[df['label'] == 'real'],df[df['label'] == 'fake']

def split_data0(df):
    train_num = 80
    valid_num = 80
    return df.iloc[:train_num],df.iloc[train_num:train_num+valid_num], df.iloc[train_num+valid_num:]
def split_data(df):
    real_data,fake_data = split_truth(df)
    train_real, valid_real,test_real = split_data0(real_data)
    train_fake, valid_fake,test_fake = split_data0(fake_data)
    train_data = pd.concat([train_real,train_fake]).sample(frac=1).reset_index(drop=True)
    valid_data = pd.concat([valid_real,valid_fake]).sample(frac=1).reset_index(drop=True)
    test_data = pd.concat([test_real,test_fake]).sample(frac=1).reset_index(drop=True)
    return train_data,valid_data,test_data


train_data,valid_data,test_data = split_data(legal_data)
train_data.shape[0],valid_data.shape[0],test_data.shape[0]

(160, 160, 14107)

In [13]:
train_data.to_csv('./train.csv',index=False,sep='\t',encoding='utf-8')
valid_data.to_csv('./valid.csv',index=False,sep='\t',encoding='utf-8')
test_data.to_csv('./test.csv',index=False,sep='\t',encoding='utf-8')